In [1]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from math import sin,cos
from __future__ import division
green = (0,255,0)
def show(image):
    plt.figure(figsize=(10,10))
    plt.imhow(image,interpolation ='nearest')

def overlay_mask(mask,image):
    rbg_mask = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    img = cv2.addWeighted(rbg_mask,0.5,image,0.5,0)
    return img

def find_biggest_contour(image):
    # copy the image
    image = image.copy()
    contours,hierarchy =cv2.findContours(image,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

    #isolating the biggest contour
    contour_size = [(cv2.contourArea(contour),contour) for contour in contours]
    biggest_contour =max(contour_size,key=lambda x:x[0])[1]
    mask = np.zeros(image.shape,np.uint8)
    cv2.drawContours(mask,[biggest_contour],-1,255,-1)
    return biggest_contour,mask
def circle_contour(image,contour):
    # bounding ellipse
    image_with_ellipse = image.copy()
    ellipse = cv2.fitEllipse(contour)
    # add it
    cv2.ellipse(image_with_ellipse,ellipse,green,2,cv2.CV_AA)
    return image_with_ellipse


def find_starrwberry(image):
    #RGB is red green blue
    #BGR is blue green red

    # step 1 convert to the correct color scheme
    image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    max_dimension =max(image.shape)

    # step 2 scale the image size property
    scale = 700/max_dimension
    image = cv2.resize(image,None,fx=scale,fy=scale)

    # step 3 clean our image
    image_blur = cv2.GaussianBlur(image,(7,7),0)
    image_blur_hsv = cv2.cvtColor(image_blur,cv2.COLOR_RGB2HSV)

    # step 4 define the filter
    # filter by the color
    min_red = np.array([0,100,80])
    max_red = np.array([10,256,256])
    mask1 = cv2.inRange(image_blur_hsv,min_red,max_red)
    min_red2 = np.array([170,100,100])
    max_red2 = np.array([180,256,256])
    mask2 = cv2.inRange(image_blur_hsv,min_red2,max_red2)
    mask = mask1 + mask2

    # step 5 segmentation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(15,15))
    mask_close = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    mask_clean = cv2.morphologyEx(mask_close,cv2.MORPH_OPEN,kernel)

    # step 6 find the biggeest starrwberry
    big_starwburry_contour,mask_starrwburries = find_biggest_contour(mask_clean)

    # step 7 overlay the masks that we created in the image
    overlay = overlay_mask(mask_clean,image)

    # step 8 circle the biggest starrwburry
    circled = circle_contour(overlay,big_starwburry_contour)
    show(circled)

    # step 9 last step that we convert back to the original color scheme
    bgr = cv2.cvtColor(circled,cv2.COLOR_BGR2RGB)
    return bgr

# read the iamge in 3 lines
image = cv2.imread('/User/tequila2411/Desktop/art.png')
result = find_starrwberry(image)
# write the new image
cv2.imwrite('/User/tequila2411/Desktop/art2.png',result)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


error: OpenCV(3.4.3) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
